In [1]:
from fastai.text.all import *


In [2]:
path = untar_data(URLs.IMDB)


In [3]:
path

Path('C:/Users/akash/.fastai/data/imdb')

In [4]:
files = get_text_files(path, folders = ['train', 'test', 'unsup'])

In [5]:
files

(#100000) [Path('C:/Users/akash/.fastai/data/imdb/test/neg/0_2.txt'),Path('C:/Users/akash/.fastai/data/imdb/test/neg/10000_4.txt'),Path('C:/Users/akash/.fastai/data/imdb/test/neg/10001_1.txt'),Path('C:/Users/akash/.fastai/data/imdb/test/neg/10002_3.txt'),Path('C:/Users/akash/.fastai/data/imdb/test/neg/10003_3.txt'),Path('C:/Users/akash/.fastai/data/imdb/test/neg/10004_2.txt'),Path('C:/Users/akash/.fastai/data/imdb/test/neg/10005_2.txt'),Path('C:/Users/akash/.fastai/data/imdb/test/neg/10006_2.txt'),Path('C:/Users/akash/.fastai/data/imdb/test/neg/10007_4.txt'),Path('C:/Users/akash/.fastai/data/imdb/test/neg/10008_4.txt')...]

In [6]:
txt = files[0].open().read()
txt

"Once again Mr. Costner has dragged out a movie for far longer than necessary. Aside from the terrific sea rescue sequences, of which there are very few I just did not care about any of the characters. Most of us have ghosts in the closet, and Costner's character are realized early on, and then forgotten until much later, by which time I did not care. The character we should really care about is a very cocky, overconfident Ashton Kutcher. The problem is he comes off as kid who thinks he's better than anyone else around him and shows no signs of a cluttered closet. His only obstacle appears to be winning over Costner. Finally when we are well past the half way point of this stinker, Costner tells us all about Kutcher's ghosts. We are told why Kutcher is driven to be the best with no prior inkling or foreshadowing. No magic here, it was all I could do to keep from turning it off an hour in."

In [7]:
spacy = WordTokenizer(buf_sz = 10)
toks = first(spacy([txt]))
WordTokenizer??

In [8]:
print(coll_repr(toks,20))

(#187) ['Once','again','Mr.','Costner','has','dragged','out','a','movie','for','far','longer','than','necessary','.','Aside','from','the','terrific','sea'...]


In [9]:
print(coll_repr(first(spacy(['hello, there my name isn\'t akash shastri\'s and I\'m very good'])),20))

(#15) ['hello',',','there','my','name','is',"n't",'akash','shastri',"'s",'and','I',"'m",'very','good'...]


In [10]:
tkn = Tokenizer(spacy)

In [11]:
print(coll_repr(tkn(txt),20))

(#207) ['xxbos','xxmaj','once','again','xxmaj','mr','.','xxmaj','costner','has','dragged','out','a','movie','for','far','longer','than','necessary','.'...]


In [12]:
txts = L(o.open(encoding = 'UTF-8').read() for o in files[:2000])

In [13]:
tok200 = txts[:200].map(tkn)

In [14]:
type(tok200)

fastcore.foundation.L

In [15]:
tok200

(#200) [(#207) ['xxbos','xxmaj','once','again','xxmaj','mr','.','xxmaj','costner','has'...],(#314) ['xxbos','xxmaj','this','is','an','example','of','why','the','majority'...],(#267) ['xxbos','xxmaj','first','of','all','i','hate','those','moronic','rappers'...],(#378) ['xxbos','xxmaj','not','even','the','xxmaj','beatles','could','write','songs'...],(#304) ['xxbos','xxmaj','brass','pictures','(','movies','is','not','a','fitting'...],(#156) ['xxbos','a','funny','thing','happened','to','me','while','watching','"'...],(#197) ['xxbos','xxmaj','this','xxmaj','german','horror','film','has','to','be'...],(#185) ['xxbos','xxmaj','being','a','long','-','time','fan','of','xxmaj'...],(#162) ['xxbos','"','tokyo','xxmaj','eyes','"','tells','of','a','17'...],(#229) ['xxbos','xxmaj','wealthy','horse','ranchers','in','xxmaj','buenos','xxmaj','aires'...]...]

In [16]:
tok200[0]

(#207) ['xxbos','xxmaj','once','again','xxmaj','mr','.','xxmaj','costner','has'...]

In [17]:
num = Numericalize()
num.setup(tok200)
coll_repr(num.vocab,20)
num??

In [18]:
folders = ['train', 'test', 'unsup']
get_data  = partial(get_text_files, folders = folders)

dls = DataBlock(
    blocks = TextBlock.from_folder(path, is_lm = True),
    get_items = get_data,
    splitter=RandomSplitter(0.1)
    ).dataloaders(path, path =path, num_workers = 0, bs = 32, seq_len = 50)

In [19]:
dls.show_batch()

,text,text_
0,"xxbos xxmaj if you want to have a clear picture of how the daily life under xxmaj stalin 's xxup ussr was , you must see this great movie . xxmaj when you watch the xxmaj russian official destroying the xxmaj french woman 's passport , you have a hint","xxmaj if you want to have a clear picture of how the daily life under xxmaj stalin 's xxup ussr was , you must see this great movie . xxmaj when you watch the xxmaj russian official destroying the xxmaj french woman 's passport , you have a hint of"
1,"for the first time in a few months , i left this movie theatre feeling i had been entertained . i now consider this movie one of my favorite foreign films . xxmaj as a testament to its greatness , i was able to show this movie to an entire","the first time in a few months , i left this movie theatre feeling i had been entertained . i now consider this movie one of my favorite foreign films . xxmaj as a testament to its greatness , i was able to show this movie to an entire suite"
2,went to hospital during filming on the film . \n\n xxmaj the movie also have some incredible fights scenes like ' ' the car park fight ' ' and ' ' the xxunk fight ranks as one of xxmaj jackie 's finest . \n\n xxmaj the movie also won award,to hospital during filming on the film . \n\n xxmaj the movie also have some incredible fights scenes like ' ' the car park fight ' ' and ' ' the xxunk fight ranks as one of xxmaj jackie 's finest . \n\n xxmaj the movie also won award for
3,too . xxmaj there`s also a few plot holes like in the opening scene involving a couple of witness protection people that the mob has caught up with . xxmaj kruger opens a car trunk and pulls out a couple of bodies and starts a fire thereby throwing the mob,. xxmaj there`s also a few plot holes like in the opening scene involving a couple of witness protection people that the mob has caught up with . xxmaj kruger opens a car trunk and pulls out a couple of bodies and starts a fire thereby throwing the mob of
4,a little annoyance only slight surprise . xxmaj dennis would have to take me kicking and screaming . xxmaj that whole section of the movie was so horrible -- especially xxmaj meg having to go to the bathroom ( # 2 ) between dumpsters in the dark at night .,little annoyance only slight surprise . xxmaj dennis would have to take me kicking and screaming . xxmaj that whole section of the movie was so horrible -- especially xxmaj meg having to go to the bathroom ( # 2 ) between dumpsters in the dark at night . e
5,"again . xxmaj but the storyline must be taken with a pinch of salt , and be prepared for an ending that most people will just be a bit dumbstruck by . xxbos xxmaj this film is brilliant ! xxmaj it touches everyone who sees it in an extraordinary way",". xxmaj but the storyline must be taken with a pinch of salt , and be prepared for an ending that most people will just be a bit dumbstruck by . xxbos xxmaj this film is brilliant ! xxmaj it touches everyone who sees it in an extraordinary way ."
6,"a change of luck . xxmaj but xxmaj sinatra 's never will ; indeed , he never does . xxmaj even while xxunk the xxunk in the new happy ending , he still seems like a self- absorbed xxunk . xxmaj his persona is the result of his character not","change of luck . xxmaj but xxmaj sinatra 's never will ; indeed , he never does . xxmaj even while xxunk the xxunk in the new happy ending , he still seems like a self- absorbed xxunk . xxmaj his persona is the result of his character not his"
7,"stall sex ( a scene so un - erotic it makes you truly wonder what anyone sees in the practice ) , the next minute he 's an expert at sexual blackmail and violent double - crossing . xxmaj this is followed by an extended scene with the character we","sex ( a scene so un - erotic it makes you truly wonder what anyone sees in the practice ) , the next minute he 's an expert at sexual blackmail and violent double - crossing . xxmaj this is followed by an extended scene with the charact

In [20]:
learn = language_model_learner(dls, AWD_LSTM, drop_mult = 0.3, metrics = [accuracy, perplexity]).to_fp16()

In [21]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.217071,4.010743,0.293764,55.187866,2:01:45


In [22]:
learn.save('1epoch')

Path('C:/Users/akash/.fastai/data/imdb/models/1epoch.pth')

In [23]:
learn = learn.load('1epoch')

In [24]:

learn.unfreeze()
learn.fit_one_cycle(6, 2e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.005681,3.870389,0.311716,47.961018,2:18:41
1,3.946382,3.811309,0.318700,45.209583,2:15:47
2,3.888460,3.740412,0.325637,42.115349,2:18:28
3,3.739059,3.666821,0.334704,39.127304,2:19:33
4,3.656766,3.614144,0.340882,37.119545,2:17:11
5,3.578729,3.603783,0.342243,36.736942,2:18:00


In [25]:

learn.save_encoder('finetuned')

In [31]:
# Testing predictions from our language model
TEXT = "I love this movie because"
N_WORDS = 80
N_SENTENCES = 1
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

print("\n".join(preds))

i love this movie because of the acting , the story , and the fact that it deals with the Australian Civil War . i was surprised to see that " sellout " has n't agreed to this role lately and i have to assume that the sort of movies he is made to do were just starting out with a desperate attempt to produce a more dramatic movie . The movie has a small Hong Kong quality ,


In [27]:
path

Path('C:/Users/akash/.fastai/data/imdb')

In [28]:
learn.export('exported_finetuned.pkl')